## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Hilo,US,19.73,-155.09,80.60,69,90,6.93,overcast clouds
1,1,Jamestown,US,42.10,-79.24,68.00,34,1,5.82,clear sky
2,2,Adre,TD,13.47,22.20,73.15,26,9,7.29,clear sky
3,3,Longyearbyen,SJ,78.22,15.64,28.40,86,20,2.24,few clouds
4,4,Hermanus,ZA,-34.42,19.23,62.01,76,0,8.01,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Hilo,US,19.73,-155.09,80.60,69,90,6.93,overcast clouds
5,5,Vaini,TO,-21.20,-175.20,78.80,74,20,6.93,few clouds
10,10,Kavieng,PG,-2.57,150.80,83.16,68,44,7.52,scattered clouds
14,14,Isangel,VU,-19.55,169.27,80.89,71,0,7.76,clear sky
18,18,Arraial Do Cabo,BR,-22.97,-42.03,75.20,69,75,6.93,broken clouds
27,27,Butaritari,KI,3.07,172.79,81.05,72,100,10.67,overcast clouds
29,29,Conceicao Da Barra,BR,-18.59,-39.73,76.05,83,0,4.74,clear sky
32,32,Ormara,PK,25.21,64.64,76.03,46,0,4.54,clear sky
38,38,Taoudenni,ML,22.68,-3.98,85.95,14,62,11.12,broken clouds
43,43,Lorengau,PG,-2.02,147.27,81.95,69,23,7.61,few clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID        0
City           0
Country        0
Lat            0
Lng            0
Max Temp       0
Humidity       0
Cloudiness     0
Wind Speed     0
Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.dropna()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Hilo,US,19.73,-155.09,80.60,69,90,6.93,overcast clouds
5,5,Vaini,TO,-21.20,-175.20,78.80,74,20,6.93,few clouds
10,10,Kavieng,PG,-2.57,150.80,83.16,68,44,7.52,scattered clouds
14,14,Isangel,VU,-19.55,169.27,80.89,71,0,7.76,clear sky
18,18,Arraial Do Cabo,BR,-22.97,-42.03,75.20,69,75,6.93,broken clouds
...,...,...,...,...,...,...,...,...,...,...
671,671,Palu,ID,-0.89,119.87,75.54,80,99,2.84,light rain
677,677,Rincon,US,32.30,-81.24,75.99,83,75,16.11,broken clouds
680,680,Santa Isabel Do Rio Negro,BR,-0.41,-65.02,75.88,90,100,1.30,moderate rain
681,681,Vanimo,PG,-2.67,141.30,76.15,93,100,8.55,heavy intensity rain


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng", "Description"]].copy() #"Current Description"

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Description,Hotel Name
0,Hilo,US,80.60,19.73,-155.09,overcast clouds,
5,Vaini,TO,78.80,-21.20,-175.20,few clouds,
10,Kavieng,PG,83.16,-2.57,150.80,scattered clouds,
14,Isangel,VU,80.89,-19.55,169.27,clear sky,
18,Arraial Do Cabo,BR,75.20,-22.97,-42.03,broken clouds,
27,Butaritari,KI,81.05,3.07,172.79,overcast clouds,
29,Conceicao Da Barra,BR,76.05,-18.59,-39.73,clear sky,
32,Ormara,PK,76.03,25.21,64.64,clear sky,
38,Taoudenni,ML,85.95,22.68,-3.98,broken clouds,
43,Lorengau,PG,81.95,-2.02,147.27,few clouds,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [9]:
# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

hotel_df

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Lat,Lng,Description,Hotel Name
0,Hilo,US,80.60,19.73,-155.09,overcast clouds,Hilo Hawaiian Hotel
5,Vaini,TO,78.80,-21.20,-175.20,few clouds,Keleti Beach Resort
10,Kavieng,PG,83.16,-2.57,150.80,scattered clouds,Nusa Island Retreat
14,Isangel,VU,80.89,-19.55,169.27,clear sky,Volcano Sea View Tree House
18,Arraial Do Cabo,BR,75.20,-22.97,-42.03,broken clouds,Pousada Porto Praia
...,...,...,...,...,...,...,...
671,Palu,ID,75.54,-0.89,119.87,light rain,Hotel Santika Palu
677,Rincon,US,75.99,32.30,-81.24,broken clouds,Bent tree subdivision
680,Santa Isabel Do Rio Negro,BR,75.88,-0.41,-65.02,moderate rain,Hotel GSA
681,Vanimo,PG,76.15,-2.67,141.30,heavy intensity rain,Vahmoneh Lodge Vanimo


In [10]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.dropna()

,City,Country,Max Temp,Lat,Lng,Description,Hotel Name
0,Hilo,US,80.60,19.73,-155.09,overcast clouds,Hilo Hawaiian Hotel
5,Vaini,TO,78.80,-21.20,-175.20,few clouds,Keleti Beach Resort
10,Kavieng,PG,83.16,-2.57,150.80,scattered clouds,Nusa Island Retreat
14,Isangel,VU,80.89,-19.55,169.27,clear sky,Volcano Sea View Tree House
18,Arraial Do Cabo,BR,75.20,-22.97,-42.03,broken clouds,Pousada Porto Praia
...,...,...,...,...,...,...,...
671,Palu,ID,75.54,-0.89,119.87,light rain,Hotel Santika Palu
677,Rincon,US,75.99,32.30,-81.24,broken clouds,Bent tree subdivision
680,Santa Isabel Do Rio Negro,BR,75.88,-0.41,-65.02,moderate rain,Hotel GSA
681,Vanimo,PG,76.15,-2.67,141.30,heavy intensity rain,Vahmoneh Lodge Vanimo


In [16]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]

fig

Figure(layout=FigureLayout(height='420px'))